# Implementing SCD Type 2 (Add New Row): 
**Maintains full history by adding new rows for changes, using start/end dates and active flags.**

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import DateType, IntegerType, LongType
from pyspark.sql.window import Window

# Get source data
df_src = spark.sql('''
select distinct(customer_key) as customer_key,
                                gender,
                                first_name,
                                last_name,
                                city,
                                zip_code,
                                birthday,
                                age,
                                vehicle_year,
                                car
  from parquet.`abfss://silver@contosoprojectstorage.dfs.core.windows.net/contoso_sales`
''')

# Initialize target DataFrame
if spark.catalog.tableExists('contoso_catalog.gold.dim_customers'):
    df_tgt = spark.sql('''
        SELECT                  customer_id, 
                                customer_key,
                                gender,
                                first_name,
                                last_name,
                                city,
                                zip_code,
                                birthday,
                                age,
                                vehicle_year,
                                car
        FROM contoso_catalog.gold.dim_customers
    ''')
else:
    df_tgt = df_src.withColumn("customer_id", F.lit(0).cast(LongType()))\
                   .withColumn("is_current", F.lit(1).cast(IntegerType()))\
                   .withColumn("start_date", F.current_date())\
                   .withColumn("end_date", F.lit(None).cast(DateType()))\
                   .filter("1 = 0")  # Empty Schema

def generate_surrogate_key(df, start_value):
    w = Window.orderBy("customer_key")
    return df.withColumn("customer_id", F.row_number().over(w) + F.lit(start_value))

def apply_scd_type2_changes(df_src, df_tgt):
    if df_tgt.rdd.isEmpty():
        return generate_surrogate_key(df_src, 0)\
               .withColumn("start_date", F.current_date())\
               .withColumn("end_date", F.lit(None).cast(DateType()))\
               .withColumn("is_current", F.lit(1).cast(IntegerType()))

    max_surrogate_key = df_tgt.agg(F.max("customer_id")).collect()[0][0]
    
    src = df_src.alias("src")
    tgt = df_tgt.filter(F.col("is_current") == 1).alias("tgt")
    
    joined_df = src.join(tgt, "customer_key", "outer")
    
    new_records = joined_df.filter(F.col("tgt.customer_id").isNull())
    
    if new_records.count() > 0:
        new_records = generate_surrogate_key(new_records, max_surrogate_key + 1)\
                     .withColumn("start_date", F.current_date())\
                     .withColumn("end_date", F.lit(None).cast(DateType()))\
                     .withColumn("is_current", F.lit(1).cast(IntegerType()))
    new_records_count = new_records.count()

    changed_records = joined_df.filter((F.col("tgt.customer_id").isNotNull()) &
                                        (       (F.coalesce(src.first_name != tgt.first_name, F.lit(False))) |
                                                (F.coalesce(src.last_name != tgt.last_name, F.lit(False))) |
                                                (F.coalesce(src.gender != tgt.gender, F.lit(False))) |
                                                (F.coalesce(src.city != tgt.city, F.lit(False))) |
                                                (F.coalesce(src.zip_code != tgt.zip_code, F.lit(False))) |
                                                (F.coalesce(src.vehicle_year != tgt.vehicle_year, F.lit(False))) |
                                                (F.coalesce(src.car != tgt.car, F.lit(False)))
                                        )
    )
    
    changed_records_count = changed_records.count()
    
    if new_records_count == 0 and changed_records_count == 0:
        return df_tgt
    
    if new_records_count > 0:
        new_records = generate_surrogate_key(new_records, max_surrogate_key + 1)\
                     .withColumn("start_date", F.current_date())\
                     .withColumn("end_date", F.lit(None).cast(DateType()))\
                     .withColumn("is_current", F.lit(1).cast(IntegerType()))
    
    if changed_records_count > 0:
        new_versions = changed_records.select(
            "src.*"
        ).drop("customer_id", "start_date", "end_date", "is_current")
        
        start_key = max_surrogate_key + new_records_count + 1
        new_versions = generate_surrogate_key(new_versions, start_key)\
                      .withColumn("start_date", F.current_date())\
                      .withColumn("end_date", F.lit(None).cast(DateType()))\
                      .withColumn("is_current", F.lit(1).cast(IntegerType()))

        old_versions = df_tgt.join(changed_records.select("customer_key"), "customer_key", "inner")\
                            .withColumn("end_date", F.when(F.col("is_current") == 1, F.date_sub(F.current_date(), 1)).otherwise(F.col("end_date")))\
                            .withColumn("is_current", F.when(F.col("is_current") == 1, F.lit(0)).otherwise(F.col("is_current")))

        unchanged_records = df_tgt.join(changed_records.select("customer_key"), "customer_key", "leftanti")
        
        # Build the final DataFrame based on what we have
        final_df = unchanged_records
        
        if changed_records_count > 0:
            final_df = final_df.unionAll(old_versions).unionAll(new_versions)
            
        if new_records_count > 0:
            final_df = final_df.unionAll(new_records)
            
        return final_df

# Apply SCD Type 2 changes
result_df = apply_scd_type2_changes(df_src, df_tgt)

display(result_df)

customer_id,customer_key,gender,first_name,last_name,city,zip_code,birthday,age,vehicle_year,car
1,11294,Female,Scarlett,Boulger,Rosebrook,2320,1952-08-20,68,2010,Ford
2,13825,Female,Victoria,Burnett,Pinbarren,4568,1973-10-19,47,2005,Maserati
3,15728,Male,Bailey,Strzelecki,Capalaba,4157,1958-09-02,62,2008,Dodge
4,19372,Male,Dylan,Wright,Smith Brook,6258,1940-02-13,80,1999,BMW
5,25048,Male,Andrew,Corlis,Sutton Forest,2577,1967-12-11,53,1998,Nissan
6,36577,Male,David,Lascelles,Bolong,2540,1955-10-06,65,2011,Maybach
7,38581,Female,Sofia,Blakeney,Wabonga,3678,1997-11-13,23,2005,Chevrolet
8,49320,Male,Hamish,Mewton-Wood,Custon,5270,2001-07-11,19,1994,Toyota
9,50915,Male,Riley,Davy,Rivett,2611,1941-07-09,79,1996,Saturn
10,51844,Male,Cody,Morrice,Eurelia,5431,1972-07-09,48,2008,Chevrolet


## Write results to table

In [0]:
result_df.write.format('delta')\
                .mode('overwrite')\
                    .option('path', 'abfss://gold@contosoprojectstorage.dfs.core.windows.net/dim_customers')\
                        .saveAsTable('contoso_catalog.gold.dim_customers')